## 1.3 UM-Bridge Client

In this section, we will implement the client-side code to interact with the UM-Bridge model server, which was defined previously [here](https://github.com/MathSEE-Modeling-Week/Modeling-Week/blob/main/UQ/Monte_Carlo/MC_server.ipynb). This will allow us to perform a Monte Carlo simulation to estimate the integral of the function $f(x) = \sin(2\pi x)$.

### MC Client Template

Below is a general template for implementing the Monte Carlo (MC) method as an UM-Bridge client. Remember that the MC estimator is given by:

\begin{equation}
\hat{F}^{MC} = \frac{1}{N} \sum_{i=1}^{N} f(x_i),
\end{equation}

where $x_i, i =1,...,N$ are realizations of the random variables $X_i$ and $f(x_i)$ represent the model evaluation for each sample $X_i, i=1,...,N$.


```python
# Monte Carlo Client Template for General MC Estimator

import numpy as np
import umbridge

# Connect to the model server
# Replace with your actual model URL and model name
model = umbridge.HTTPModel("http://<server-address>:<port>", "<model-name>")

# Define the number of samples for the Monte Carlo simulation
N = <number_of_samples>

# Sample X_i from a distribution
# Replace <distribution> with the distribution you consider for MC
X_i_samples = [<distribution>(<param_1>, <param_2>, <param_size>).tolist() for _ in range(N)]
# Example: X_i_samples = [np.random.uniform(0, 1, 1).tolist() for _ in range(N)]

# Monte Carlo simulation: Evaluate the model for each sample X_i -> f(X_i)
mc_values = [model([X_i])[0][0] for X_i in X_i_samples]  # Modify as needed

# Compute the Monte Carlo estimator: \hat{F}^{MC} = (1/N) * sum(f(X_i))
mc_estimator = np.mean(mc_values)
print("Monte Carlo Estimator (mean of evaluations):", mc_estimator)
```

---

### MC Client for Solving an Integral

Next, we will implement the MC client to compute the value of the integral:

\begin{equation}
\int_{0}^{1} f(x) \textit{d}x.
\end{equation}

From basic analysis, we know that for $f(x) = \sin(2\pi x)$ the exact value of this integral is $0$. To estimate this integral using MC simulation, we sample $N$ uniformly distributed random variables $X_i \sim \mathscr{U}([0,1])$, $i=1,...,N$. 

Since

$\mathbb{E}[f(X_i)] = \int_{0}^{1} f(x) \textit{d}x$,

the choice of the distribution for $X_i$ provides an unbiased estimator for the integral.

In [2]:
# umbridge client
import numpy as np
import umbridge

model = umbridge.HTTPModel("http://localhost:4242", "forward")

N = 100 # Number of samples

# Monte Carlo simulation

# Generate N random samples from a uniform distribution over [0, 1] and convert each to a list
parameters = [np.random.rand(1).tolist() for _ in range(N)]
print("First ten parameters:", parameters[:10], "\n") # print first 10 parameters

mc_values = [model([parameters])[0][0] for parameters in parameters]  # model evaluation for each parameter
print("First ten model evaluations:", mc_values[:10], "\n")

mc_mean = np.mean(mc_values)  # calculate mean
print("MC estimator:", mc_mean, "\n")

First ten parameters: [[0.8706080117991248], [0.13244494965600662], [0.5076992710333947], [0.2428955294556927], [0.1120323797472873], [0.38042373882307357], [0.3434180589147875], [0.956555804923446], [0.4806179552064962], [0.8981198774492936]] 

First ten model evaluations: [-0.7263481727585729, 0.7393982614219202, -0.04835708034929064, 0.99900385843762, 0.6472110655448976, 0.6826038543936931, 0.8326265266319042, -0.26959066053474917, 0.12148018789807949, -0.5973010634556816] 

MC estimator: -0.0531669564566899 



### Understanding the Results

- **Monte Carlo Estimator**: The estimator uses random samples from the uniform distribution $[0, 1]$ and evaluates the UM-Bridge model $f(x) = \sin(2\pi x)$ for each sample. The estimator is the mean of these evaluations.
- **Sample Size \( N \)**: The accuracy of the Monte Carlo method depends on the number of samples $N$. Increasing $N$ generally improves the estimate's accuracy, though the method converges slowly with an error proportional to $\mathcal{O}(N^{-1/2})$.

---

### Tasks

You can experiment with the MC method by adjusting the following:

1. **Change the Sample Size**: Try increasing $N$ to improve the accuracy of the estimator and observe how the result changes.
2. **Change the Function**: To estimate the integral of another function, modify the `Testmodel` class in the model server. Change the definition of `posterior` to evaluate a different 1D function of your interest.

#### Remarks

- The **convergence** of the basic Monte Carlo method is dimension-independent, meaning it works in any dimension. However, the convergence rate is relatively slow at $\mathcal{O}(N^{-1/2})$, making it inefficient for high-dimensional problems.
- **Quasi-Monte Carlo (QMC)** methods can improve convergence to $\mathcal{O}(N^{-1})$, but they typically require smoothness in the function being integrated. QMC methods are also dimension-independent, though they perform better in low to moderate dimensions.
